In [16]:
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np

In [17]:
batch_size = 1
path = "~/dev/news_sentiment/data/"

In [18]:
#select important data
data = pd.read_csv(path+"news_train_from2013.csv")
data_trimmed = data[['headline', "sentimentClass"]]
data_trimmed = data_trimmed.dropna()

#split into training and test
msk = np.random.rand(len(data_trimmed)) < 0.01
train = data_trimmed[msk]
test=data_trimmed[~msk]

train.to_csv(path+"training.csv")
test.to_csv(path+"test.csv")

In [19]:
len(test)

9162230

In [20]:
len(train)

92560

In [21]:
data_lm = TextDataBunch.from_csv(path, "training.csv", text_cols = "headline", label_cols = "sentimentClass").split_by_rand_pct(0.1, seed=69).label_for_lm().databunch(bs=batch_size, num_workers = 1)

In [22]:
data_lm.show_batch()

idx,text
0,24 0 xxunk 1942 36 xxunk 92 239 ] xxbos [ 2 6 xxunk 6 xxunk 6 148 6 301 9 6 xxunk 42 6 80 131 6 149 6 xxunk 6 20 6 xxunk 86 6 189 6 xxunk 6 15 14 0 6 104 6 231 ] xxbos [ 2 6 141 6 116 6 xxunk 6 409 6 2034 6 17 6 xxunk 6 284 6 59


In [23]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [24]:
wiki_itos = pickle.load(open(Config().model_path()/'wt103-fwd/itos_wt103.pkl', 'rb'))
vocab = data_lm.vocab
vocab.stoi["finance"]

481

In [25]:
len(wiki_itos)

60000

In [26]:
len(vocab.itos)

27320

In [27]:
i, unks = 0, []
while len(unks) < 50:
    if data_lm.vocab.itos[i] not in wiki_itos: unks.append((i,data_lm.vocab.itos[i]))
    i += 1
wiki_words = set(wiki_itos)
news_words = set(vocab.itos)
wiki_not_news = wiki_words.difference(news_words)
wiki_not_news_list = []
for i in range(100):
    word = wiki_not_news.pop()
    wiki_not_news_list.append(word)
    wiki_not_news.add(word)
wiki_not_news_list[:15]

['marengo',
 'euboea',
 'copernicus',
 'woolly',
 'bana',
 'aerobics',
 'portraiture',
 'mummies',
 'clarisse',
 'arcminutes',
 'tabla',
 'ruthlessness',
 'unicameral',
 'bedingfield',
 'polis']

In [28]:
news_not_wiki = news_words.difference(wiki_words)
news_not_wiki_list = []
for i in range(100):
    word = news_not_wiki.pop()
    news_not_wiki_list.append(word)
    news_not_wiki.add(word)
news_not_wiki_list[:15]

['aptargroup',
 'illumina',
 'headcount',
 'tsl.n',
 'synovis',
 'kemet',
 'afl.n',
 '2-conocophillips',
 'nsm.n',
 'curtailment',
 'reg-3i',
 'teliasonera',
 'carmike',
 'mitek',
 'lawmaker']